### Stack Overflow: Using NLP to find answers to top unanswered question

In [0]:
!pip install nltk

     |████████████████████████████████| 1.5 MB 5.2 MB/s 
     |████████████████████████████████| 77 kB 7.6 MB/s 
     |████████████████████████████████| 768 kB 11.4 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-3ba20400-e4c0-42cb-9e1a-e69a506766ff/bin/python -m pip install --upgrade pip' command.


In [0]:
!pip install beautifulsoup4

     |████████████████████████████████| 142 kB 5.7 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-3ba20400-e4c0-42cb-9e1a-e69a506766ff/bin/python -m pip install --upgrade pip' command.


In [0]:
#doanloading nltk library
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Out[3]: True

In [0]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
Out[4]: True

In [0]:
from pyspark.sql.functions import sum, regexp_replace, col, explode, desc, split, year, lit
from pyspark.sql.types import IntegerType, FloatType, DataType, TimestampType, StringType, StructType, StructField, ArrayType
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from bs4 import BeautifulSoup
import string
import re
import matplotlib.pyplot as plt
from collections import defaultdict
import math


#### Importing files into dataframes

In [0]:
from pyspark.sql.functions import sum
from pyspark.sql.types import IntegerType, FloatType, DataType, TimestampType, StringType, StructType, StructField


# File location
filename1 = "/FileStore/tables/Assignment1/SO_ML.csv"
filename2 = "/FileStore/tables/Assignment1/SO_Spark.csv"
filename3 = "/FileStore/tables/Assignment1/SO_Security.csv"

#Function to import csv files
def import_csv(filename):
     #creating schema to ensure files are in appropriate format.
    new_Schema = StructType([ StructField("Id", IntegerType(), True),
                      StructField("PostTypeId", IntegerType(), True),
                      StructField("AcceptedAnswerId", IntegerType(), True),
                      StructField("ParentId", IntegerType(), True),
                       StructField("CreationDate", TimestampType(), True),
                       StructField("DeletionDat", TimestampType(), True),
                       StructField("Score", IntegerType(), True),
                       StructField("ViewCount", IntegerType(), True),
                       StructField("Body", StringType(), True),
                       StructField("OwnerUserId", IntegerType(), True),
                       StructField("OwnerDisplayName", StringType(), True),
                       StructField("LastEditorUserId", IntegerType(), True),
                       StructField("LastEditorDisplayName", StringType(), True),
                       StructField("LastEditDate", TimestampType(), True),
                       StructField("LastActivityDate", TimestampType(), True),
                       StructField("Title", StringType(), True),
                       StructField("Tags", StringType(), True),
                       StructField("AnswerCount", IntegerType(), True),
                       StructField("CommentCount", IntegerType(), True),
                       StructField("FavoriteCount", IntegerType(), True),
                       StructField("ClosedDate", TimestampType(), True),
                       StructField("CommunityOwnedDate", TimestampType(), True),
                       StructField("ContentLicense", StringType(), True)
                      ])


    # CSV options
    infer_schema = "false"
    first_row_is_header = "true"
    delimiter = ","
    multiLine = "true"
    escape = "\""

    # CSV options
    file_type = "csv"
    infer_schema = "true"
    first_row_is_header = "true"
    delimiter = ","
    
    #loading file
    df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .option("multiLine", multiLine) \
  .option("escape", escape) \
  .schema(new_Schema) \
  .load("/FileStore/tables/Assignment1/"+filename)
    
    return df    

#function to perform column arithmetic (row count and sum of column values)
def column_sum(df, col_name):
    #Using count() to count the number of rows
    row_count = df.count()
    print("The total number of ViewCount in the file is: " + str(row_count))
    #Using sum() to calculate the sum of the ViewCounts and displaying it.
    df.select(sum(col_name)).show()

#assigning the names of the files into variables
file_ML = "SO_ML.csv"
file_Spark = "SO_Spark.csv"
file_Security = "SO_Security.csv"

#importing each file
df_ML = import_csv(file_ML)
df_Spark = import_csv(file_Spark)
df_Security = import_csv(file_Security)

In [0]:
#function to perform column arithmetic (row count and sum of column values)
def column_sum(df, col_name):
    #Using count() to count the number of rows
    row_count = df.count()
    print("The total number of ViewCount in the file is: " + str(row_count))
    #Using sum() to calculate the sum of the ViewCounts and displaying it.
    df.select(sum(col_name)).show()

print(file_ML)
column_sum(df_ML, "ViewCount")

print(file_Spark)
column_sum(df_Spark, "ViewCount")

print(file_Security)
column_sum(df_Security, "ViewCount")

SO_ML.csv
The total number of ViewCount in the file is: 50000
+--------------+
|sum(ViewCount)|
+--------------+
|     102120351|
+--------------+

SO_Spark.csv
The total number of ViewCount in the file is: 50000
+--------------+
|sum(ViewCount)|
+--------------+
|     140958653|
+--------------+

SO_Security.csv
The total number of ViewCount in the file is: 50000
+--------------+
|sum(ViewCount)|
+--------------+
|     163597256|
+--------------+



### Preprocessing of data in Files

In [0]:
#Merging columns
@udf
def column_merger(col1, col2):
    newCol= col1 + " " + col2
    return newCol
    
df_ML_Combined = df_ML.select("*",column_merger("Title","Body").alias("TitleBody"))
df_Spark_Combined = df_Spark.select("*",column_merger("Title","Body").alias("TitleBody"))
df_Security_Combined = df_Security.select("*",column_merger("Title","Body").alias("TitleBody"))
#display(df_ML_Combined)

In [0]:
#parsing html text in words
@udf
def preProcessText(body):
    newBody=""
    #soups = BeautifulSoup(body, "html.parser").find_all("p")
    soups = BeautifulSoup(body, "html.parser")
    
    #code tag is removed
    for data in soups(['code']):
        data.decompose()
    
    #content of remainder tags extracted
    for d in soups:
        newBody+=d.get_text().lower()
    return newBody
    

#Parsing all data files
newML = df_ML_Combined.select("*",preProcessText("TitleBody").alias("newBody"))
newSpark = df_Spark_Combined.select("*",preProcessText("TitleBody").alias("newBody"))
newSecurity = df_Security_Combined.select("*",preProcessText("TitleBody").alias("newBody"))
#display(newML)

In [0]:
#Function to remove stopwords and lemmatization
@udf
def removeStopWords(sentences):
    lemmatizer = WordNetLemmatizer()
    #stemmer = nltk.stem.SnowballStemmer()
    noStopWords = []
    stopword = stopwords.words('english')
    for sentence in sent_tokenize(sentences): #breaking into sentences
        for word in word_tokenize(sentence): #breaking into words
            if word not in stopword and len(word)>2: #removing stopwords and other irrelevant words
                if "n't" not in word: #because words like "doest" appear before doesn't leaving alot of "n't"
                    noStopWords.append(lemmatizer.lemmatize(word))
                    #noStopWords.append(stemmer.stem(word))
    return ' '.join(noStopWords)

#Removing stopwords from files
newMLnonstop1 = newML.select("*", removeStopWords("newBody").alias("noStopWords"))
newSparknonstop1 = newSpark.select("*", removeStopWords("newBody").alias("noStopWords"))
newSecuritynonstop1 = newSecurity.select("*", removeStopWords("newBody").alias("noStopWords"))

In [0]:
#UDF to remove puntuations and non-alphabets
@udf
def removePuncNonalphabets(column):
    
    #using regex to extract only alphabets and whitespaces
    return re.sub('[^a-zA-Z\s]+', "",column)
    
#removing punctuation and non-alphabets
filtered_ML = newMLnonstop1.withColumn("cleanWords", removePuncNonalphabets("noStopWords"))
filtered_Spark = newSparknonstop1.withColumn("cleanWords", removePuncNonalphabets("noStopWords"))
filtered_Security = newSecuritynonstop1.withColumn("cleanWords", removePuncNonalphabets("noStopWords"))

In [0]:
#getting most unanswered questions
def maxUnansweredQuestion(df):
    #Extracting unaswered questions
    unanswered = df.filter(df['AcceptedAnswerId'].isNull())
    sortedUnanswered = unanswered.sort("Score", ascending=False)
    return sortedUnanswered.first(), sortedUnanswered

#
maxUnansweredML, unanswered_ML = maxUnansweredQuestion(filtered_ML)
maxUnansweredSpark, unanswered_Spark = maxUnansweredQuestion(filtered_Spark)
maxUnansweredSecurity, unanswered_Security = maxUnansweredQuestion(filtered_Security)

In [0]:
#display(unanswered_Spark)

In [0]:
print("Max Score in ML")
print(maxUnansweredML['cleanWords'])
print("\nMax Score in Spark")

print(maxUnansweredSpark['cleanWords'])

print("\nMax Score in Security")
print(maxUnansweredSecurity['cleanWords'])

Max Score in ML
epoch iteration training neural network difference epoch iteration training multilayer perceptron

Max Score in Spark
spark javalangoutofmemoryerror java heap space cluster master slave node memory setting problem first read data  hdfs rdd second something rdd last output hdfs run program show many task every thing input data  solve problem

Max Score in Security
tokenbased authentication want understand tokenbased authentication mean searched internet could find anything understandable


In [0]:
#UDF to calculate cosine similarity
@udf
def cosineSimilarity(posAnswer, unaccepted):
    #combining unanswered question and possible similar question
    dwords = list(set(unaccepted.split() + posAnswer.split()))
    
    dt1 = []
    dt2 = []
    
    for word in dwords:
        if word in posAnswer:
            dt1.append(1)
        else:
            dt1.append(0)
    
        if word in maxUn:
            dt2.append(1)
        else:
            dt2.append(0)
    
    #counter = 0
    dotProduct = 0
    t1Summer = 0
    t2Summer = 0
    
    for i in range(0,len(dt1)):
    #while(counter< len(dt1)):
        dotProduct += dt1[i]*dt2[i]
        t1Summer += dt1[i]*dt1[i]
        t2Summer += dt2[i]*dt2[i]
        #counter+=1
    
    root = math.sqrt(t1Summer*t2Summer)
    
    if root ==0:
        return 0
    
    return (dotProduct/root)


similarity_ML = filtered_ML.filter(filtered_ML["AcceptedAnswerId"].isNotNull()).withColumn("similarity",cosineSimilarity(col("cleanWords"),lit(maxUnansweredML['cleanWords']))).sort("similarity",ascending=False)

similarity_Spark = filtered_Spark.filter(filtered_Spark["AcceptedAnswerId"].isNotNull()).withColumn("similarity",cosineSimilarity(col("cleanWords"),lit(maxUnansweredSpark['cleanWords']))).sort("similarity",ascending=False)

similarity_Security = filtered_Security.filter(filtered_Security["AcceptedAnswerId"].isNotNull()).withColumn("similarity",cosineSimilarity(col("cleanWords"),lit(maxUnansweredSecurity['cleanWords']))).sort("similarity",ascending=False)  

In [0]:
#display(similarity_ML)

In [0]:
#display(similarity_Spark)

In [0]:
#display(similarity_Security)

In [0]:
#Selecting the top 3 rows with the most similar questions
Top3_ML = similarity_ML.take(3)
Top3_Spark = similarity_Spark.take(3)
Top3_Security = similarity_Security.take(3)

In [0]:
print("**********************ML****************************")
print("The unanswered question with the highest score in ML: ")
print("Id = {} \n {}".format(maxUnansweredML['Id'], maxUnansweredML['newBody']))
print("The top 3 most similar questions with accepted answers:")
print("1. Id = {} \nSimilarity Score: {}\nQuestion: {} \n".format(Top3_ML[0]['Id'],Top3_ML[0]['similarity'], Top3_ML[0]['newBody'] ))
print("2. Id = {} \nSimilarity Score: {}\nQuestion: {} \n".format(Top3_ML[1]['Id'],Top3_ML[1]['similarity'], Top3_ML[1]['newBody']))
print("3. Id = {} \nSimilarity Score: {}\nQuestion: {} \n".format(Top3_ML[2]['Id'],Top3_ML[2]['similarity'], Top3_ML[2]['newBody']))

**********************ML****************************
The unanswered question with the highest score in ML: 
Id = 4752626 
 epoch vs iteration when training neural networks what is the difference between epoch and iteration when training a multi-layer perceptron?

The top 3 most similar questions with accepted answers:
1. Id = 46621774 
Similarity Score: 0.5892556509887896
Question: epochs vs pass vs iteration what does the term epochs mean in the neural network.
how does it differ from pass and iteration
 

2. Id = 60691599 
Similarity Score: 0.4472135954999579
Question: sudden drop in validation accuracy during training when i was training my neural network there is a sudden drop in validation accuracy during the 8th epoch what does this mean?

 

3. Id = 69808910 
Similarity Score: 0.4472135954999579
Question: why does the result from evaluate() differ from last epoch result? i have a very simple neural-network which works in 250 epochs and in the last epoch it shows the  , however, 

In [0]:
print("**********************SPARK****************************")
print("The unanswered question with the highest score in Spark: ")
print("Id = {} \n{}".format(maxUnansweredSpark['Id'], maxUnansweredSpark['newBody']))
print("\n\nThe top 3 most similar questions with accepted answers:\n")
print("1. Id = {} \nSimilarity Score: {} \n{}\n".format(Top3_Spark[0]['Id'],Top3_Spark[0]['similarity'], Top3_Spark[0]['newBody']))
print("2. Id = {} \nSimilarity Score: {} \n{}\n".format(Top3_Spark[1]['Id'], Top3_Spark[1]['similarity'], Top3_Spark[1]['newBody']))
print("3. Id = {} \nSimilarity Score: {} \n{}\n".format(Top3_Spark[2]['Id'], Top3_Spark[2]['similarity'], Top3_Spark[2]['newBody']))

**********************SPARK****************************
The unanswered question with the highest score in Spark: 
Id = 21138751 
spark java.lang.outofmemoryerror: java heap space my cluster: 1 master, 11 slaves, each node has 6 gb memory.
my settings:

here is the problem:
first, i read some data (2.19 gb) from hdfs to rdd:

second, do something on this rdd:

last, output to hdfs:

when i run my program it shows:

there are too many tasks?
ps: every thing is ok when the input data is about 225 mb. 
how can i solve this problem?



The top 3 most similar questions with accepted answers:

1. Id = 49304799 
Similarity Score: 0.4117055577420104 
java spark collect() javardd fails with memory errors (emr cluster) im running spark app on aws emr(elastic map reduce) cluster
my master node characteristics are: 8 vcore, 15 gib memory, 80 ssd gb storage
my executors node are: 8 vcore, 15 gib memory, 80 ssd gb storage
i have csv inputfile  file with size - 600mb.i am trying to read it to javardd 

In [0]:
print("******************SECURITY**************************")
print("The unanswered question with the highest score in Security: ")
print("Id = {} \n{}".format(maxUnansweredSecurity['Id'], maxUnansweredSecurity['cleanWords']))
print("\n\nThe top 3 most similar questions with accepted answers:\n")
print("1. Id = {} \nSimilarity Score: {} \n{}\n".format(Top3_Security[0]['Id'], Top3_Security[0]['similarity'],Top3_Security[0]['newBody']))
print("2. Id = {} \nSimilarity Score: {} \n{}\n".format(Top3_Security[1]['Id'], Top3_Security[1]['similarity'], Top3_Security[1]['newBody']))
print("3. Id = {} \nSimilarity Score: {} \n{}\n".format(Top3_Security[2]['Id'], Top3_Security[2]['similarity'], Top3_Security[2]['newBody']))

******************SECURITY**************************
The unanswered question with the highest score in Security: 
Id = 1592534 
tokenbased authentication want understand tokenbased authentication mean searched internet could find anything understandable


The top 3 most similar questions with accepted answers:

1. Id = 62015970 
Similarity Score: 0.4082482904638631 
what does -nd argument of ssh mean? what does the -nd argument of ssh mean? i didn't manage to find it on the internet.
example:



2. Id = 37314142 
Similarity Score: 0.36563621206356534 
security mechanism in wsdl i want to know how wsdl secure data on trafic. i searched but i can't find anything i need. this is a wsdl service sample and i want to understand their security mechanism.

this 

is encode data trafic right ?


3. Id = 4750377 
Similarity Score: 0.3481553119113957 
authentication by delegation: what does it mean? what does "authentication by delegation" mean?


